In [1]:
import sys
import os
import fastText

from validation import compute_f1

from keras.models import load_model
from keras.callbacks import ModelCheckpoint, Callback

import models
import utils

/home/gwiedemann/miniconda3/envs/kerasenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [3]:
trainSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-train.tsv')
devSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-dev.tsv')
testSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-test.tsv')

# trainSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.train.bio')
# devSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.testa.bio')
# testSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.testb.bio')

print(len(trainSentences))
print(len(devSentences))
print(len(testSentences))


24000
2200
5100


In [4]:
print(testSentences[3])

[['Die', 'O'], ['These', 'O'], [',', 'O'], ['Schlatter', 'B-PER'], ['sei', 'O'], ['Antisemit', 'O'], ['gewesen', 'O'], [',', 'O'], ['wurde', 'O'], ['seither', 'O'], ['in', 'O'], ['der', 'O'], ['theologischen', 'O'], ['Fachliteratur', 'O'], ['nicht', 'O'], ['mehr', 'O'], ['vertreten', 'O'], ['.', 'O']]


In [5]:
labelSet = set()
characters= set()
models.max_sequence_length = 0

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for word, label in sentence:
            for char in word:
                characters.add(char)
            labelSet.add(label)
        if len(sentence) > models.max_sequence_length:
            models.max_sequence_length = len(sentence)

In [6]:
print(len(labelSet))
print(models.max_sequence_length)

25
56


In [7]:
# :: Create a mapping for the labels ::
models.label2Idx = {"PADDING_TOKEN":0}
for label in labelSet:
    models.label2Idx[label] = len(models.label2Idx)

In [8]:
print(models.label2Idx)

{'I-OTH': 18, 'I-LOCderiv': 19, 'I-PERderiv': 1, 'B-PERderiv': 20, 'B-OTH': 3, 'PADDING_TOKEN': 0, 'I-PER': 4, 'B-PERpart': 21, 'O': 5, 'B-ORGderiv': 25, 'B-OTHderiv': 7, 'I-LOCpart': 10, 'B-LOC': 8, 'I-OTHderiv': 9, 'B-ORGpart': 11, 'I-ORG': 2, 'B-PER': 13, 'I-PERpart': 6, 'B-LOCderiv': 14, 'B-LOCpart': 22, 'B-OTHpart': 23, 'I-OTHpart': 24, 'I-ORGderiv': 15, 'I-ORGpart': 12, 'I-LOC': 16, 'B-ORG': 17}


In [9]:
# :: Hard coded case lookup ::
models.case2Idx = {'PADDING_TOKEN':0, 'numeric': 1, 'allLower':2, 'allUpper':3, 'initialUpper':4, 'other':5, 'mainly_numeric':6, 'contains_digit': 7}

In [10]:
print(models.case2Idx)

{'contains_digit': 7, 'allLower': 2, 'other': 5, 'initialUpper': 4, 'allUpper': 3, 'numeric': 1, 'PADDING_TOKEN': 0, 'mainly_numeric': 6}


In [11]:
print(type(trainSentences))

<class 'list'>


In [12]:
print(trainSentences[0])

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [13]:
models.char2Idx={"PADDING_TOKEN":0, "<S>":1, "</S>":2, "<W>":3, "</W>":4}
for char in characters:
    models.char2Idx[char] = len(models.char2Idx)
models.char2Idx['UNKNOWN'] = len(models.char2Idx)
print(models.char2Idx)

{'є': 6, 'Ü': 7, '©': 9, 'ī': 10, 'C': 5, '守': 12, '別': 16, '章': 14, '¹': 17, 'Т': 18, '³': 19, '柯': 20, 'g': 22, 'м': 23, 'п': 8, 'ν': 263, '›': 25, 'õ': 28, 's': 29, 'ŏ': 30, 'J': 31, 'Ø': 219, 'y': 32, 'Ц': 34, 'ņ': 249, '!': 35, '公': 36, 'G': 217, 'è': 37, 'ı': 11, 'X': 39, '懿': 40, 'Ö': 242, 'κ': 41, 'ς': 42, '5': 44, 'S': 45, 'a': 46, '8': 47, '”': 13, 'à': 48, 'Ş': 304, 'д': 321, 'オ': 49, 'й': 50, 'č': 198, 'ɨ': 51, 'H': 52, '</S>': 2, 'ę': 53, 'V': 61, '=': 54, 'Ž': 55, 'ō': 117, 'н': 256, ']': 188, 'r': 57, 'ó': 318, '3': 271, 'ś': 298, 'L': 59, 'z': 60, '\x95': 62, '術': 302, 'α': 63, 'ǒ': 64, 'ệ': 65, 'ñ': 66, '+': 67, 'г': 68, '"': 69, '‹': 70, 'K': 71, 'ô': 72, 'ψ': 74, 'я': 322, 'ğ': 224, 'б': 75, 'Á': 173, '冲': 331, '*': 76, '€': 191, 'с': 77, '太': 78, 'ż': 292, 'Z': 79, '≤': 80, 'İ': 114, '}': 81, '\x9a': 82, 'd': 83, 'A': 84, '鷹': 87, '“': 86, 'ã': 88, '寝': 91, 'm': 24, '<S>': 1, 'š': 93, '7': 316, 'É': 95, 'UNKNOWN': 333, 'á': 96, 'ế': 97, '鶴': 21, 'η': 183, '算': 98, '

In [14]:
print(trainSentences[0]) 

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [15]:
models.ft = fastText.load_model("../embeddings/wiki.de.bin")

In [16]:
print(models.nb_embedding_dims)
print(len(trainSentences[0]))

300
25


In [17]:
models.idx2Label = {v: k for k, v in models.label2Idx.items()}
print(len(models.label2Idx))
print(len(models.idx2Label))

26
26


# Test Model

In [ ]:
# import importlib
# importlib.reload(models)

In [ ]:
tmp_model_filename = 'model_lstm_germeval.h5'
# checkpoint = ModelCheckpoint(tmp_model_filename, verbose=1, save_best_only = True, monitor = 'val_acc')
history = utils.F1History(tmp_model_filename, devSet = devSentences)
model = models.get_model_lstm()

In [ ]:
#import importlib
#importlib.reload(utils)
# print(models.max_sequence_length)

In [ ]:
model.fit_generator(
    utils.NerSequence(trainSentences[:1000], shuffle_data=True, batch_size=256), 
    validation_data = utils.NerSequence(devSentences[:1000], batch_size=256), 
    epochs = 3, callbacks = [history]
)

In [ ]:
print(history.acc)
print(history.f1_scores)

In [ ]:
model.load_weights(tmp_model_filename)

In [ ]:
model.fit_generator(
    utils.NerSequence(trainSentences, shuffle_data=True, batch_size=2048), 
    validation_data = utils.NerSequence(devSentences, batch_size=256), 
    epochs = 5, callbacks = [history]
)

In [ ]:
model.load_weights(tmp_model_filename)

In [ ]:
true_labels, pred_labels = utils.predict_sequences(model, testSentences[:1000])
print(compute_f1(pred_labels, true_labels, models.idx2Label))

# Experiments

In [ ]:
f = open('results_lstm_germeval.txt', 'a')
for run_i in range(10):
    print("Run " + str(run_i))
    
    tmp_model_filename = 'model_lstm_germeval.' + str(run_i) + '.h5'

    history = utils.F1History(tmp_model_filename, devSet=devSentences)

    model = models.get_model_lstm()
    model.fit_generator(
        utils.NerSequence(trainSentences, shuffle_data=True, batch_size=16), 
        validation_data = utils.NerSequence(devSentences, batch_size=512), 
        epochs = 10, callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    
    model.fit_generator(
        utils.NerSequence(trainSentences, shuffle_data=True, batch_size=512), 
        validation_data = utils.NerSequence(devSentences, batch_size=512), 
        epochs = 10, callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    
    true_labels, pred_labels = utils.predict_sequences(model, testSentences)
    
    pre, rec, f1 = compute_f1(pred_labels, true_labels, models.idx2Label)
    f.write(str(run_i) + "\t" + str(pre) + "\t" + str(rec) +  "\t" + str(f1))
    f.write("\n")
    f.flush()
f.close()

Run 0
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
casing_input (InputLayer)       (None, None)         0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10688       char_input[0][0]                 
__________________________________________________________________________________________________
words_input (InputLayer)        (None, None, 300)    0                                            
__________________________________________________________________________________________________
case

Epoch 1/10
1499/1500 [============================>.] - ETA: 0s - loss: 0.0401 - acc: 0.9838
New maximum F1 score: 0.772098385857033 (before: 0) Saving to model_lstm_germeval.1.h5
1500/1500 [==============================] - 650s 433ms/step - loss: 0.0401 - acc: 0.9838 - val_loss: -0.0031 - val_acc: 0.9893
Epoch 2/10
1499/1500 [============================>.] - ETA: 0s - loss: -0.0247 - acc: 0.9888
New maximum F1 score: 0.790446841294299 (before: 0.772098385857033) Saving to model_lstm_germeval.1.h5
1500/1500 [==============================] - 641s 427ms/step - loss: -0.0248 - acc: 0.9888 - val_loss: -0.0528 - val_acc: 0.9902
Epoch 3/10
1499/1500 [============================>.] - ETA: 0s - loss: -0.0786 - acc: 0.9901
New maximum F1 score: 0.8038585209003216 (before: 0.790446841294299) Saving to model_lstm_germeval.1.h5
1500/1500 [==============================] - 641s 428ms/step - loss: -0.0786 - acc: 0.9901 - val_loss: -0.1064 - val_acc: 0.9906
Epoch 4/10
1499/1500 [=================

1499/1500 [============================>.] - ETA: 0s - loss: -0.1843 - acc: 0.9917
New maximum F1 score: 0.8194110020374142 (before: 0.8114522508947071) Saving to model_lstm_germeval.2.h5
1500/1500 [==============================] - 634s 423ms/step - loss: -0.1844 - acc: 0.9917 - val_loss: -0.2095 - val_acc: 0.9911
Epoch 6/10
1499/1500 [============================>.] - ETA: 0s - loss: -0.2394 - acc: 0.9922
New maximum F1 score: 0.8207878675790641 (before: 0.8194110020374142) Saving to model_lstm_germeval.2.h5
1500/1500 [==============================] - 638s 426ms/step - loss: -0.2394 - acc: 0.9922 - val_loss: -0.2630 - val_acc: 0.9912
Epoch 7/10
1500/1500 [==============================] - 639s 426ms/step - loss: -0.2942 - acc: 0.9926 - val_loss: -0.3164 - val_acc: 0.9909
Epoch 8/10
1500/1500 [==============================] - 638s 426ms/step - loss: -0.3486 - acc: 0.9930 - val_loss: -0.3701 - val_acc: 0.9907
Epoch 9/10
1499/1500 [============================>.] - ETA: 0s - loss: -0.

# Save final model

In [ ]:
import shutil, json
# copy file for best run
shutil.copyfile('tmp_3cnn_bi-lstm.h5', '../models/final_model_germeval_outer.h5')
with open("../models/final_model_germeval.indexes", "w") as f:
    json.dump([models.idx2Label, models.label2Idx, models.char2Idx, models.case2Idx], f)

In [ ]:
from keras_contrib.layers import CRF
def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "loss": loss, "accuracy":accuracy}

finalmodel = load_model('../models/final_model_germeval_outer.h5', custom_objects=create_custom_objects())
true_labels, pred_labels = utils.predict_sequences(finalmodel, testSentences)
print(compute_f1(pred_labels, true_labels, models.idx2Label))

In [ ]:
f = open('germeval_output.tsv', 'w', encoding='UTF-8')
for i_sent, sent in enumerate(testSentences):
    for i_tok, tok in enumerate(sent):
        if tok[0] == 'PADDING_TOKEN':
            break
        correctlabel = models.idx2Label[true_labels[i_sent][i_tok]]
        guessedlabel = models.idx2Label[pred_labels[i_sent][i_tok]]
        line = "\t".join([str(i_tok+1), tok[0], correctlabel, correctlabel, guessedlabel, guessedlabel])
        f.write(line + '\n')
    f.write('\n')
f.close